In [8]:
import base64
from Crypto import Random
from Crypto.Cipher import AES
from Crypto.Protocol.KDF import PBKDF2

BS = 128
pad = lambda s: s + (BS - len(s) % BS) * chr(BS - len(s) % BS)
unpad = lambda s : s[0:-s[-1]]


class AESCipher:

    def __init__( self, key ):
        self.key = key.encode('utf-8')
        #hashlib.sha256(key.encode('utf-8')).digest()

    def encrypt( self, raw ):
        raw = pad(raw)
        iv = Random.new().read( AES.block_size )
        cipher = AES.new( self.key, AES.MODE_CBC, iv )
        return base64.b64encode( iv + cipher.encrypt( raw ) )

    def decrypt( self, enc ):
        enc = base64.b64decode(enc)        
        # first 32 bytes are salt
        salt = enc[:32]
        # generate password using key and salt, 32 block size, 1000 iters        
        password = PBKDF2(self.key, salt, 32, 1000)
        # take first 32 bytes as the key
        keyBytes = password[:32]
        # use key and first 16 of password for the IV
        cipher = AES.new(keyBytes, AES.MODE_CBC, password[:16] )
        #return unpad(cipher.decrypt( enc[32:] ))
        return unpad(cipher.decrypt(enc[32:]))


cipher = AESCipher('deloitte')
file = open("DeloitteEncrypted.txt", "r")
txt = file.read()
decrypted = cipher.decrypt(txt)




FileNotFoundError: [Errno 2] No such file or directory: 'DeloitteEncrypted.txt'

NameError: name 'decrypted' is not defined